In [1]:
import mlflow
from train import load_data, preprocess_data, run_experiment
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [2]:
mlflow.set_experiment("SMS Spam Classification")
    
# Load data
X_train, y_train = load_data('processed_data/train.csv')
X_val, y_val = load_data('processed_data/validation.csv')
X_test, y_test = load_data('processed_data/test.csv')

# Preprocess data
data = preprocess_data(X_train, X_val, X_test)
X_train_tfidf = data['X_train_tfidf']
X_val_tfidf = data['X_val_tfidf']
X_test_tfidf = data['X_test_tfidf']

# Define models
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': LinearSVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42)
}

2025/03/04 16:07:17 INFO mlflow.tracking.fluent: Experiment with name 'SMS Spam Classification' does not exist. Creating a new experiment.


In [3]:
# Run experiments for each model
results = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    _, test_metrics = run_experiment(
        model_name, model, 
        X_train_tfidf, y_train, 
        X_val_tfidf, y_val, 
        X_test_tfidf, y_test
    )
    results[model_name] = test_metrics

Training Naive Bayes...


2025/03/04 16:07:46 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Training SVM...


2025/03/04 16:08:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Training Logistic Regression...


2025/03/04 16:08:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [4]:
# Print results
print("\nModel Performance Metrics on Test Data:")
print("\nClassifier\t\t\tAccuracy\tF1 Score\tRecall\t\tPrecision\tSpecificity\tROC AUC\t\tPR AUC")
print("-" * 120)

for model_name, metrics in results.items():
    print(f"{model_name:<20}\t\t{metrics['accuracy']:.4f}\t\t{metrics['f1_score']:.4f}\t\t"
            f"{metrics['recall']:.4f}\t\t{metrics['precision']:.4f}\t\t{metrics['specificity']:.4f}\t\t"
            f"{metrics['roc_auc']:.4f}\t\t{metrics['pr_auc']:.4f}")

# Select the best model based on PR AUC
best_model = max(results.items(), key=lambda x: x[1]['pr_auc'])
print(f"\nBest model based on PR AUC: {best_model[0]} with PR AUC = {best_model[1]['pr_auc']:.4f}")


Model Performance Metrics on Test Data:

Classifier			Accuracy	F1 Score	Recall		Precision	Specificity	ROC AUC		PR AUC
------------------------------------------------------------------------------------------------------------------------
Naive Bayes         		0.9570		0.8273		0.7055		1.0000		1.0000		0.8528		0.9428
SVM                 		0.9803		0.9290		0.8834		0.9796		0.9968		0.9401		0.9676
Logistic Regression 		0.9570		0.8286		0.7117		0.9915		0.9989		0.8553		0.9531

Best model based on PR AUC: SVM with PR AUC = 0.9676


In [12]:
client = mlflow.tracking.MlflowClient()
experiment_id = "431075443274287447"
runs = client.search_runs(
    experiment_id, order_by=["metrics.pr_auc ASC"]
)
for run in runs:
    print(run.info.run_name, run.data.metrics['test_pr_auc'])


Logistic Regression 0.9530785765562111
SVM 0.9675831829013415
Naive Bayes 0.9427791772476878
